<a href="https://colab.research.google.com/github/DelcioLuis/AdminTemnoappe-/blob/master/facenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install facenet-pytorch
!pip install albumentations

# !pip install tensorflow==2.12.0 # Certifique-se de instalar a versão correta do TensorFlow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Define function-helpers for data preprocessing**

**Define image path**



In [3]:
import os
import glob
import re

from PIL import Image, ImageDraw


def get_files(path='./', ext=('.png', '.jpeg', '.jpg')):
    """ Get all image files and sort them by the number in their name """
    files = []
    for e in ext:
        files.extend(glob.glob(f'{path}/**/*{e}', recursive=True))
    files.sort(key=lambda p: (os.path.dirname(p), extract_number(os.path.basename(p))))
    return files

def extract_number(filename):
    """ Extract number from filename, default to 0 if no number is found """
    match = re.search(r'(\d+)(?=\.\w+$)', filename)  # Find number before the extension
    return int(match.group(1)) if match else 0


def to_rgb_and_save(path):
    """ Some of the images may have RGBA mode """
    for p in path:
        img = Image.open(p)
        if img.mode != 'RGB':
            img = img.convert('RGB')
            img.save(p)

In [4]:
ABS_PATH = '/content/drive/My Drive/treinomodelo/'
DATA_PATH = os.path.join(ABS_PATH, 'data')

TRAIN_DIR = os.path.join(DATA_PATH, 'treino')
TEST_DIR = os.path.join(DATA_PATH, 'teste')

ALIGNED_TRAIN_DIR = TRAIN_DIR + '_cropped'
ALIGNED_TEST_DIR = TEST_DIR + '_cropped'

**Preparing data**

In [5]:
from collections import Counter


# 1. Get path for TRAIN_DIR/TEST_DIR
trainF, testF = get_files(TRAIN_DIR), get_files(TEST_DIR)

# prepare info for printing
trainC, testC = Counter(map(os.path.dirname, trainF)), Counter(map(os.path.dirname, testF))
train_total, train_text  = sum(trainC.values()), '\n'.join([f'\t- {os.path.basename(fp)} - {c}' for fp, c in trainC.items()])
test_total, test_text  = sum(testC.values()), '\n'.join([f'\t- {os.path.basename(fp)} - {c}' for fp, c in testC.items()])

print(f'Train files\n\tpath: {TRAIN_DIR}\n\ttotal number: {train_total}\n{train_text}')
print(f'Train files\n\tpath: {TEST_DIR}\n\ttotal number: {test_total}\n{test_text}')

Train files
	path: /content/drive/My Drive/modelos/data/treino
	total number: 3945
	- Abah Festus - 8
	- Abay Tsehaye - 8
	- Abdelaziz Bouteflika - 8
	- Abdelaziz Djerad - 8
	- Abdelmadjid Tebboune - 8
	- Abebaw butako - 8
	- Abedi Pele - 8
	- Abel Edime - 8
	- Abosede Olajide - 8
	- Abubeker Nassir - 8
	- Abuya Solomon - 8
	- Adam Bombolb - 8
	- Adam Ounas - 8
	- Adane Girma - 8
	- Adebimpe Olarewaju - 8
	- Adedini Adeola - 8
	- Adegoke Akinyele - 8
	- Adegoke Olamide - 8
	- Adem Zorgane - 8
	- Adesua Etomi - 8
	- Adigun Pelumi - 8
	- Agostinho Kapaia - 8
	- Ahmed Ben Bella - 8
	- Ahmed Elmohamady - 8
	- Ahmed Hegazi - 8
	- Ahmed Kathrada - 8
	- Ahmed Magdy - 8
	- Ahmed Mahgoub - 8
	- Ahmed Mazouz - 8
	- Ahmed Sayed - 8
	- Ahmed saad - 8
	- Ajewole Fasanmi Richard - 8
	- Ajewole Olaitan - 8
	- Akua Dansua - 8
	- Akwa - 8
	- Aladeokin A. Abiodun - 8
	- Albert Luthuli - 8
	- Alemu Aga - 8
	- Alula Girma - 8
	- Amin Askar - 8
	- AminB - 8
	- Amira Selim - 8
	- Amos Wako - 8
	- Amr Waked 

In [ ]:
# 2. Convert all img to RGB
to_rgb_and_save(trainF), to_rgb_and_save(testF)

**Image plotter fuctions**

In [ ]:
from math import ceil

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
%matplotlib inline

from matplotlib.patches import Ellipse


def imshow(img, ax, title):
    ax.imshow(img)
    if title:
        el = Ellipse((2, -1), 0.5, 0.5)
        ax.annotate(title, xy=(1, 0), xycoords='axes fraction', ha='right', va='bottom',
                    bbox=dict(boxstyle="round", fc="0.8"),
                    arrowprops=dict(arrowstyle="simple", fc="0.6", ec="none",
                                    patchB=el, connectionstyle="arc3, rad=0.3"))
    ax.set_xticks([]), ax.set_yticks([])

def plot_gallery(images, ncols, nrows, titles=None, title='', figsize=None):
    if figsize is None:
        figsize = (18, ncols) if ncols < 10 else (18, 20)
    fig = plt.figure(figsize=figsize)
    grid = ImageGrid(fig, 111, nrows_ncols=(nrows, ncols), axes_pad=0.02)

    for i, ax in enumerate(grid):
        if i == len(images): break
        imshow(images[i], ax, titles[i] if titles is not None else '')

    # there are some problems with suptitle alignment
    y_title_pos = grid[0].get_position().get_points()[1][1] - 0.33 / (1 if nrows == 1 else nrows / 3)
    plt.suptitle(title, y=y_title_pos, fontsize=12)

def plot(paths=None, images=None, titles=None, axtitle=True, title='', to_size=(512, 512)):
    """
    Plot image gallery by passing (paths, title) or (images, titles)
    :param paths: list of image paths
    :param images: list of (PIL.Image | np.array | torch.Tensor) objects
    :param titles: list of image titles
    :param bool axtitle: if paths is not None, then axtitle=True leads to use basedir name as titles
    :param str title: gallery title
    :param to_size: image resizing size before plot, default (512, 512)
    """

    if paths is not None and len(paths):
        images = [Image.open(p).resize(to_size) for p in paths]

        nrows = int(ceil(len(images) / 12)) # 12 images per row
        ncols = 12 if nrows > 1 else len(images)

        if axtitle:
              titles = [os.path.dirname(p).split('/')[-1] for p in paths]

        plot_gallery(images, ncols, nrows, titles, title)

    elif images is not None and len(images):
        if isinstance(images, list):
            images = np.array(images)

        nrows = int(ceil(len(images) / 12)) # 12 images per row
        ncols = 12 if nrows > 1 else len(images)

        # Rescale to [0., 1.]
        if images[0].max() > 1:
            images /= 255.

        # if torch.Tensor change axes
        if not isinstance(images, np.ndarray):
            if images.size(1) == 3 or 1:
                images = images.permute((0, 2, 3, 1))

        plot_gallery(images, ncols, nrows, titles, title)


    else:
        raise LookupError('You didnt pass any path or image objects')
    plt.show()

In [ ]:
plot(paths=trainF, title='Train images')

In [ ]:
plot(paths=testF, title='Test images')

**Install facenet_pytorch with MTCNN detection and pretrained vggface-2 InceptionResnetV1**

In [ ]:
# !pip install facenet-pytorch

from facenet_pytorch import MTCNN, InceptionResnetV1, training, fixed_image_standardization

**Function for cropping and saving images based on MTCNN detector**

In [ ]:
import tqdm

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np

import shutil

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {device}')


def crop_face_and_save(path, new_path=None, model=MTCNN, transformer=None, params=None):
    """
    Detect face on each image, crop them and save to "new_path"
    :param str path: path with images will be passed to  datasets.ImageFolder
    :param str new_path: path to locate new "aligned" images, if new_path is None
                     then new_path will be path + "_cropped"
    :param model: model to detect faces, default MTCNN
    :param transformer: transformer object will be passed to ImageFolder
    :param params: parameters of MTCNN model
    """
    if not new_path:
        new_path = path + '_cropped'

    # in case new_path exists MTCNN model will raise error
    if os.path.exists(new_path):
        shutil.rmtree(new_path)

    # it is default parameters for MTCNN
    if not params:
        params = {
            'image_size': 160, 'margin': 0,
            'min_face_size': 10, 'thresholds': [0.6, 0.7, 0.7],
            'factor': 0.709, 'post_process': False, 'device': device
            }

    model = model(**params)

    if not transformer:
        transformer = transforms.Lambda(
            lambd=lambda x: x.resize((1280, 1280)) if (np.array(x) > 2000).all() else x
        )
    # for convenience we will use ImageFolder instead of getting Image objects by file paths
    dataset = datasets.ImageFolder(path, transform=transformer)
    dataset.samples = [(p, p.replace(path, new_path)) for p, _ in dataset.samples]

    # batch size 1 as long as we havent exact image size and MTCNN will raise an error
    loader = DataLoader(dataset, batch_size=1, collate_fn=training.collate_pil)
    for i, (x, y) in enumerate(tqdm.tqdm(loader)):
        model(x, save_path=y)

    # spare some memory
    del model, loader, dataset

**Detect/Crop faces and save "new" aligned images**

In [ ]:
# 3. Crop train dataset faces and save aligned images
print('\t- Train data')
crop_face_and_save(TRAIN_DIR, ALIGNED_TRAIN_DIR)

detect_failed_train_files = []

# check if some imgs were missed by detector and failed to save
train_files, train_aligned_files = get_files(TRAIN_DIR), get_files(ALIGNED_TRAIN_DIR)
if len(train_files) != len(train_aligned_files):
    files = set(map(lambda fp: os.path.relpath(fp, start=TRAIN_DIR), train_files))
    aligned_files = set(map(lambda fp: os.path.relpath(fp, start=ALIGNED_TRAIN_DIR), train_aligned_files))
    detect_failed_train_files = list(files - aligned_files)
    print(f"\nfiles {len(aligned_files)}/{len(files)}: {', '.join(detect_failed_train_files)} were not saved")

# -------------                     -------------

# Crop test dataset faces and save aligned images
print('\t- Test data')
crop_face_and_save(TEST_DIR, ALIGNED_TEST_DIR)

# check if some imgs were missed by detector and failed to save
test_files, test_aligned_files = get_files(TEST_DIR), get_files(ALIGNED_TEST_DIR)
if len(test_files) != len(test_aligned_files):
    files = set(map(lambda fp: os.path.relpath(fp, start=TEST_DIR), test_files))
    aligned_files = set(map(lambda fp: os.path.relpath(fp, start=ALIGNED_TEST_DIR), test_aligned_files))
    detect_failed_test_files = list(files - aligned_files)
    print(f"\nfiles {len(aligned_files)}/{len(files)}: {', '.join(detect_failed_train_files)} were not saved")

In [ ]:
trainF = get_files(ALIGNED_TRAIN_DIR)
plot(paths=trainF, title='Aligned train images')

In [ ]:
testF = get_files(ALIGNED_TEST_DIR)
plot(paths=testF, title='Aligned test images')

In [ ]:
# Plot failed to aligned train images
trainFailF = list(map(lambda fp: os.path.join(TRAIN_DIR, fp), detect_failed_train_files))

if trainFailF:  # Verifique se há imagens para plotar
    plot(paths=trainFailF)
else:
    print("Nenhuma imagem falhou no alinhamento, nada a plotar.")

**Install albumentations for augmentations**

In [ ]:
# !pip install albumentations

import albumentations as A

**Transformer for data**

In [ ]:
from facenet_pytorch import fixed_image_standardization

standard_transform = transforms.Compose([
                                np.float32,
                                transforms.ToTensor(),
                                fixed_image_standardization
])

aug_mask = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.15),
    A.RandomBrightnessContrast(contrast_limit=0.5, brightness_limit=0.2, p=0.4),
    A.Rotate(30, p=0.2),
    A.RandomSizedCrop((120, 120), 160, 160, p=0.4),
    A.OneOrOther(A.ImageCompression(p=0.2), A.Blur(p=0.2), p=0.66),
    A.OneOf([
        A.Rotate(45, p=0.3),
        A.ElasticTransform(sigma=20, alpha_affine=None, border_mode=0, p=0.2)  # alpha_affine definido como None
    ], p=0.5),
    A.HueSaturationValue(val_shift_limit=10, p=0.3)
], p=1)


transform = {
    'train': transforms.Compose([
                                 transforms.Lambda(lambd=lambda x: aug_mask(image=np.array(x))['image']),
                                 standard_transform
    ]),
    'test': standard_transform
}

**DataLoader for train/test**

In [ ]:
from torch.utils.data import SubsetRandomSampler


b = 32

# Original train images
trainD = datasets.ImageFolder(ALIGNED_TRAIN_DIR, transform=standard_transform)
# Augmented train images
trainD_aug = datasets.ImageFolder(ALIGNED_TRAIN_DIR, transform=transform['train'])
# Train Loader
trainL = DataLoader(trainD, batch_size=b, num_workers=2) # x: torch.Size([batch_size, 3, 160, 160]), y: torch.Size([batch_size])
trainL_aug = DataLoader(trainD_aug, batch_size=b, num_workers=2)

# Original test images
testD = datasets.ImageFolder(ALIGNED_TEST_DIR, transform=standard_transform)
# Test Loader
testL = DataLoader(testD, batch_size=b, num_workers=2)

# Convert encoded labels to named claasses
IDX_TO_CLASS = np.array(list(trainD.class_to_idx.keys()))
CLASS_TO_IDX = dict(trainD.class_to_idx.items())


**Prepare model**

In [ ]:
from facenet_pytorch import InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2', dropout_prob=0.5, device=device).eval()

**Function for embedding extraction**

In [ ]:
def fixed_denormalize(image):
    """ Restandartize images to [0, 255]"""
    return image * 128 + 127.5

def getEmbeds(model, n, loader, imshow=False, n_img=5):
    model.eval()
    # images to display
    images = []

    embeds, labels = [], []
    for n_i in tqdm.trange(n):
        for i, (x, y) in enumerate(loader, 1):

            # on each first batch get 'n_img' images
            if imshow and i == 1:
                inds = np.random.choice(x.size(0), min(x.size(0), n_img))
                images.append(fixed_denormalize(x[inds].data.cpu()).permute((0, 2, 3, 1)).numpy())

            embed = model(x.to(device))
            embed = embed.data.cpu().numpy()
            embeds.append(embed), labels.extend(y.data.cpu().numpy())

    if imshow:
        plot(images=np.concatenate(images))

    return np.concatenate(embeds), np.array(labels)

**Extract embeddings**

In [ ]:
# 3. Get embeddings
# Train embeddings
trainEmbeds, trainLabels = getEmbeds(model, 1, trainL, False)
trainEmbeds_aug, trainLabels_aug = getEmbeds(model, 50, trainL_aug, imshow=True, n_img=3)

trainEmbeds = np.concatenate([trainEmbeds, trainEmbeds_aug])
trainLabels = np.concatenate([trainLabels, trainLabels_aug])

# Test embeddings
testEmbeds, testLabels = getEmbeds(model, 1, testL, False)

In [ ]:
# 4. Save embeddings
TRAIN_EMBEDS = os.path.join(DATA_PATH, 'trainEmbeds.npz')
TEST_EMBEDS = os.path.join(DATA_PATH, 'testEmbeds.npz')

np.savez(TRAIN_EMBEDS, x=trainEmbeds, y=trainLabels)
np.savez(TEST_EMBEDS, x=testEmbeds, y=testLabels)

# Load the saved embeddings to use them futher
trainEmbeds, trainLabels = np.load(TRAIN_EMBEDS, allow_pickle=True).values()
testEmbeds, testLabels = np.load(TEST_EMBEDS, allow_pickle=True).values()

# Get named labels
trainLabels, testLabels = IDX_TO_CLASS[trainLabels], IDX_TO_CLASS[testLabels]

**Function for embedding calculations**

In [ ]:
from sklearn.metrics import pairwise_distances
import pandas as pd
import seaborn as sns
sns.set()

def getDist(x, metric='euclidean', index=None, columns=None):
    dists = pairwise_distances(x, x, metric=metric)
    return pd.DataFrame(dists, index=index, columns=columns)

def heatmap(x, title='', cmap='Greens', linewidth=1):
    plt.figure(figsize=(17, 12))
    plt.title(title)
    sns.heatmap(x, cmap=cmap, square=True)
    plt.show()

**Get distance matrix for each image embedding**

In [ ]:
num_originals = min(88, len(trainEmbeds))  # Garante que não ultrapassamos o tamanho real de trainEmbeds
inds = range(num_originals)

# Calcular distâncias usando apenas os índices válidos
# Embeddings de treino - distância euclidiana
dists = getDist(trainEmbeds[inds], metric='euclidean', index=trainLabels[inds], columns=trainLabels[inds])
heatmap(dists, 'euclidean distance')

# Embeddings de treino - distância de cosseno
dists = getDist(trainEmbeds[inds], metric='cosine', index=trainLabels[inds], columns=trainLabels[inds])
heatmap(dists, 'cosine distance')

In [ ]:
# Test embeddings
dists = getDist(testEmbeds, metric='euclidean', index=testLabels, columns=testLabels)
heatmap(dists, 'euclidean distance')

dists = getDist(testEmbeds, metric='cosine', index=testLabels, columns=testLabels)
heatmap(dists, 'cosine distance')

**Take a look at how TSNE & PCA methods clustered our images**

In [ ]:
from sklearn.manifold import TSNE


# Determine o número de amostras disponíveis
num_train_samples = min(88, len(trainEmbeds))  # Use o menor valor entre 88 e o tamanho de trainEmbeds
inds = range(num_train_samples)  # Atualize o intervalo de índices

# Agora use o índice atualizado para t-SNE
X_tsne1 = TSNE(n_components=2, init='pca', random_state=33).fit_transform(trainEmbeds[inds])
X_tsne2 = TSNE(n_components=2, init='random', random_state=33).fit_transform(trainEmbeds[inds])
y = [CLASS_TO_IDX[label] for label in trainLabels[inds]]

fig, ax = plt.subplots(1, 2, figsize=(12, 8))

img = ax[0].scatter(X_tsne1[:, 0], X_tsne1[:, 1], c=y, alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral', 10))
ax[1].scatter(X_tsne2[:, 0], X_tsne2[:, 1], c=y, alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral', 10))

ax[0].set_title('TSNE with pca init')
ax[1].set_title('TSNE with random init')
plt.suptitle('Face embeddings')

cbar = plt.colorbar(img, ax=ax)
cbar.ax.set_yticklabels(np.unique(trainLabels[inds]))
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# Obtenha o número de amostras
n_samples = trainEmbeds.shape[0]  # Total de amostras em trainEmbeds

# Use o número de amostras para definir o intervalo, garantindo que não ultrapasse
inds = range(min(88, n_samples))  # Garante que não ultrapasse o tamanho

# PCA
X_pca1 = PCA(n_components=2, random_state=33).fit_transform(trainEmbeds[inds])
y = [CLASS_TO_IDX[label] for label in trainLabels[inds]]

fig, ax = plt.subplots(1, 1, figsize=(12, 8))
img = ax.scatter(X_pca1[:, 0], X_pca1[:, 1], c=y, alpha=0.5, cmap=plt.cm.get_cmap('nipy_spectral', 10))

plt.title('PCA method')
plt.suptitle('Face embeddings')

cbar = plt.colorbar(img, ax=ax)
cbar.ax.set_yticklabels(np.unique(trainLabels[inds]))
plt.show()


**Find optimal parameters for SVC classifier and train**

In [ ]:
# data preparation
X = np.copy(trainEmbeds)
y = np.array([CLASS_TO_IDX[label] for label in trainLabels])

print(f'X train embeds size: {X.shape}')
print(f'Tagret train size: {y.shape}')

**As we see, in order to find optimal parameters among ```'param_grid'``` the whole search process took ~ 1.5 hours**

In [ ]:
%%time

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
import warnings

warnings.filterwarnings('ignore', 'Solver terminated early.*')

# Preparação dos dados
X = np.copy(trainEmbeds)  # Supondo que trainEmbeds esteja definido no arquivo principal
y = np.array([CLASS_TO_IDX[label] for label in trainLabels])  # Supondo que CLASS_TO_IDX e trainLabels estejam definidos

# Verificação das classes
unique_classes = np.unique(y)
print("Classes únicas em y:", unique_classes)
print("Número de classes:", len(unique_classes))

if len(unique_classes) <= 1:
    raise ValueError("O conjunto de dados deve conter mais de uma classe para treinar o modelo.")

# Espaço de hiperparâmetros com mais opções para aumentar a precisão
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],  # Ampliado para incluir valores menores
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],  # Ampliado para incluir valores maiores
    'kernel': ['linear', 'rbf']  # Incluído 'linear' para testar outro tipo de kernel
}

# Parâmetros adicionais do modelo SVM
model_params = {'class_weight': 'balanced', 'probability': True, 'random_state': 3}

# Criando o modelo SVC
model = SVC(**model_params)

# Busca exaustiva com GridSearchCV
clf = GridSearchCV(model, param_grid, cv=5, verbose=3, n_jobs=-1)  # 'cv=5' para validação cruzada, 'n_jobs=-1' usa todos os núcleos disponíveis
clf.fit(X, y)

# Resultados
print('Melhor estimador: ', clf.best_estimator_)
print('Melhores parâmetros: ', clf.best_params_)
print('Acurácia com os melhores parâmetros: {:.2f}%'.format(clf.best_score_ * 100))


**Load & save SVC model, basically weights**

In [ ]:
import joblib
import os

# Definir o caminho para salvar o modelo SVM
SVM_PATH = os.path.join(DATA_PATH, 'svm.sav')

# Verificar se o modelo já existe. Se não existir, treina e salva o modelo.
if os.path.exists(SVM_PATH):
    print("Carregando o modelo salvo.")
    clf = joblib.load(SVM_PATH)
else:
    print("Modelo não encontrado. Treinando e salvando o modelo.")

    # Parâmetros do SVM e do GridSearch
    param_grid = {'C': [1, 10, 100, 1e3, 5e3, 1e4, 5e4, 1e5],
                  'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 'auto'],
                  'kernel': ['rbf', 'sigmoid', 'poly']}
    model_params = {'class_weight': 'balanced', 'max_iter': 10, 'probability': True, 'random_state': 3}

    # Inicializar o modelo SVM e GridSearchCV
    model = SVC(**model_params)
    clf = GridSearchCV(model, param_grid)

    # Treinar o modelo
    clf.fit(X, y)

    # Salvar o modelo treinado
    joblib.dump(clf, SVM_PATH)
    print("Modelo salvo em:", SVM_PATH)

# Exibir os melhores parâmetros do modelo
print('Melhor estimador: ', clf.best_estimator_)
print('Melhores parâmetros: ', clf.best_params_)


**Check the accuracy score on Train & Test datasets(embeddings)**

In [ ]:
# test data preparation
X_test, y_test = np.copy(testEmbeds), np.array([CLASS_TO_IDX[label] for label in testLabels])
print(f'X train embeds size: {X_test.shape}')
print(f'Tagret train size: {y_test.shape}')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

inds = range(88)
train_acc = accuracy_score(clf.predict(X[inds]), y[inds])
print(f'Accuracy score on train data: {train_acc:.3f}')

test_acc = accuracy_score(clf.predict(X_test), y_test)
print(f'Accuracy score on test data: {test_acc}')

**Some functional for video preprocessing**

In [ ]:
import cv2
import imageio

def toGif(path, dim):
    gpath = ''.join(path.split('.')[:-1]) + '.gif'

    with imageio.get_writer(gpath, mode='I') as writer:
        frames = []
        capture = cv2.VideoCapture(path)

        i = 0
        while True:
            ret, frame = capture.read()
            if not ret: break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            writer.append_data(cv2.resize(image, dim))
            i += 1
        print(f'Total frames: {i}')


In [ ]:
# Create gifs from the existing videos in our path
VIDEO_PATH = os.path.join(DATA_PATH, 'videos/')
width, height = 640, 360

mov1 = os.path.join(VIDEO_PATH, '1.mp4')
toGif(mov1, (width, height))

mov2 = os.path.join(VIDEO_PATH, '2.mp4')
toGif(mov2, (width, height))

**Bounding boxes can overlap, so define some function to avoid duplicates**

In [ ]:
def diag(x1, y1, x2, y2):
    return np.linalg.norm([x2 - x1, y2 - y1])

def square(x1, y1, x2, y2):
    return abs(x2 - x1) * abs(y2 - y1)

def isOverlap(rect1, rect2):
    x1, x2 = rect1[0], rect1[2]
    y1, y2 = rect1[1], rect1[3]

    x1_, x2_ = rect2[0], rect2[2]
    y1_, y2_ = rect2[1], rect2[3]

    if x1 > x2_ or x2 < x1_: return False
    if y1 > y2_ or y2 < y1_: return False

    rght, lft = x1 < x1_ < x2, x1_ < x1 < x2_
    d1, d2 = 0, diag(x1_, y1_, x2_, y2_)
    threshold = 0.5

    if rght and y1 < y1_: d1 = diag(x1_, y1_, x2, y2)
    elif rght and y1 > y1_: d1 = diag(x1_, y2_, x2, y1)
    elif lft and y1 < y1_: d1 = diag(x2_, y1_, x1, y2)
    elif lft and y1 > y1_: d1 = diag(x2_, y2_, x1, y1)

    if d1 / d2 >= threshold and square(x1, y1, x2, y2) < square(x1_, y1_, x2_, y2_): return True
    return False

# Função principal para desenhar caixas
def draw_box(draw, boxes, names, probs, min_p=0.95):
    font = ImageFont.truetype(os.path.join(ABS_PATH, 'arial.ttf'), size=22)

    not_overlap_inds = []
    for i in range(len(boxes)):
        not_overlap = True
        for box2 in boxes:
            if np.all(boxes[i] == box2): continue
            not_overlap = not isOverlap(boxes[i], box2)
            if not not_overlap: break
        if not_overlap: not_overlap_inds.append(i)

    boxes = [boxes[i] for i in not_overlap_inds]
    probs = [probs[i] for i in not_overlap_inds]
    for box, name, prob in zip(boxes, names, probs):
        if prob >= min_p:
            draw.rectangle(box.tolist(), outline=(255, 255, 255), width=5)
            x1, y1, _, _ = box
            # Usando getbbox para obter a largura e altura do texto
            bbox = font.getbbox(f'{name}: {prob:.2f}')
            text_width, text_height = bbox[2] - bbox[0], bbox[3] - bbox[1]
            draw.rectangle(((x1, y1 - text_height), (x1 + text_width, y1)), fill='white')
            draw.text((x1, y1 - text_height), f'{name}: {prob:.2f}', (24, 12, 30), font)

    return boxes, probs

**Functional for processing video**

In [ ]:
standard_transform = transforms.Compose([
                                transforms.Resize((160, 160)),
                                np.float32,
                                transforms.ToTensor(),
                                fixed_image_standardization
])

def get_video_embedding(model, x):
    embeds = model(x.to(device))
    return embeds.detach().cpu().numpy()


def is_known_face(embedding, known_embeddings, threshold=0.8):
    similarities = cosine_similarity([embedding], known_embeddings)
    return similarities.max() >= threshold

# def face_extract(model, clf, frame, boxes):
#     names, prob = [], []
#     if len(boxes):
#         x = torch.stack([standard_transform(frame.crop(b)) for b in boxes])
#         embeds = get_video_embedding(model, x)
#         idx, prob = clf.predict(embeds), clf.predict_proba(embeds).max(axis=1)
#         names = [IDX_TO_CLASS[idx_] for idx_ in idx]
#     return names, prob

def face_extract(model, clf, frame, boxes, known_embeddings, threshold=0.8):
    names, probs = [], []
    if len(boxes):
        x = torch.stack([standard_transform(frame.crop(b)) for b in boxes])
        embeds = get_video_embedding(model, x)  # Extrai as embeddings para cada rosto detectado

        # Verifique se cada embedding corresponde a uma pessoa conhecida
        for embed in embeds:
            if is_known_face(embed, known_embeddings, threshold):
                idx = clf.predict([embed])[0]
                prob = clf.predict_proba([embed]).max()
                names.append(IDX_TO_CLASS[idx])
                probs.append(prob)
            else:
                # Marque como "Desconhecido" se a pessoa não for reconhecida
                names.append("Desconhecido")
                probs.append(0.0)
    return names, probs

known_embeddings = trainEmbeds

def preprocess_image(detector, face_extractor, clf, path, transform=None):
    if not transform: transform = lambda x: x.resize((1280, 1280)) if (np.array(x.size) > 2000).all() else x
    capture = Image.open(path).convert('RGB')
    i = 0

    # iframe = Image.fromarray(transform(np.array(capture)))
    iframe = transform(capture)

    boxes, probs = detector.detect(iframe)
    if boxes is None: boxes, probs = [], []
    names, prob = face_extract(face_extractor, clf, iframe, boxes, known_embeddings, threshold=0.8)

    frame_draw = iframe.copy()
    draw = ImageDraw.Draw(frame_draw)

    boxes, probs = draw_box(draw, boxes, names, probs)
    return frame_draw.resize((620, 480), Image.BILINEAR)




def preprocess_video(detector, face_extractor, clf, path, transform=None, k=3):
    frames = []
    if not transform: transform = lambda x: x.resize((1280, 1280)) if (np.array(x.shape) > 2000).all() else x
    capture = cv2.VideoCapture(path)
    i = 0
    while True:
        ret, frame = capture.read()
        if not ret: break

        iframe = Image.fromarray(transform(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

        if (i + 1) % k:
            boxes, probs = detector.detect(iframe)
            if boxes is None: boxes, probs = [], []
            names, prob = face_extract(face_extractor, clf, iframe, boxes, known_embeddings, threshold=0.8)

        frame_draw = iframe.copy()
        draw = ImageDraw.Draw(frame_draw)

        boxes, probs = draw_box(draw, boxes, names, probs)
        frames.append(frame_draw.resize((620, 480), Image.BILINEAR))
        i += 1

    print(f'Total frames: {i}')
    return frames

def framesToGif(frames, path):
    with imageio.get_writer(path, mode='I') as writer:
        for frame in tqdm.tqdm(frames):
            writer.append_data(np.array(frame))

**Define our models and parameters**

In [ ]:
from PIL import ImageFont


k = 3 # each k image will be processed by networks
font = ImageFont.truetype(os.path.join(ABS_PATH, 'arial.ttf'), size=22)

mtcnn = MTCNN(keep_all=True, min_face_size=70, device=device)
model = InceptionResnetV1(pretrained='vggface2', dropout_prob=0.6, device=device).eval()

**Process video and save to gif**

# > Add blockquote



In [ ]:
%%time
print('Processing mov1: ')
frames = preprocess_video(mtcnn, model, clf, mov1)
mov1_aug = os.path.join(VIDEO_PATH, '1.gif')
framesToGif(frames, mov1_aug)

In [ ]:
%%time
print('Processing mov2: ')
frames = preprocess_video(mtcnn, model, clf, mov2)
mov2_aug = os.path.join(VIDEO_PATH, '2.gif')
framesToGif(frames, mov2_aug)

**Some examples**

![image](https://drive.google.com/uc?export=view&id=15fSxB0Z9lUInaTgEHsFqgYp5XdkbVTRd)

![image](https://drive.google.com/uc?export=view&id=1KHCNLFnM4vCxWAEB78CzN-3nQMTTAdqc)

**Some photos**

In [ ]:
ADD_DATA = os.path.join(DATA_PATH, 'images')

frame = preprocess_image(mtcnn, model, clf, os.path.join(ADD_DATA, '1.jpeg'))
frame.save(os.path.join(ADD_DATA, '1_augs'), 'gif')

![image](https://drive.google.com/uc?export=view&id=1KTpgQFNH7aBn6ru7e6zR8RLyNy_onL6G)


In [ ]:
ADD_DATA = os.path.join(DATA_PATH, 'images')

frame = preprocess_image(mtcnn, model, clf, os.path.join(ADD_DATA, '2.jpeg'))
frame.save(os.path.join(ADD_DATA, '2_aug'), 'gif')

In [ ]:
!pip install tensorflow==2.12.0 # Certifique-se de instalar a versão correta do TensorFlow

  Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.22.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
facenet-pytorch 2.6.0 requires numpy<2.0.0,>=1.24.0, but you have numpy 1.23.5 which is incompatible.
tf-keras 2.17.0

In [ ]:
import joblib
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Carregar o modelo SVM treinado
svm_model = joblib.load(SVM_PATH)  # Usa o caminho salvo como `SVM_PATH`

# Dividir os embeddings e rótulos em treino e teste
X_train, X_test, y_train, y_test = train_test_split(trainEmbeds, trainLabels, test_size=0.2, random_state=42)

# Obter previsões do SVM para que a rede neural aprenda a imitar o SVM
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)


print("y_train_pred", y_train_pred)

print('y_test_pred', y_test_pred)

# Converter rótulos para categóricos (necessário para a rede neural)
y_train_cat = to_categorical(y_train_pred)
y_test_cat = to_categorical(y_test_pred)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Definir uma rede neural simples
model_nn = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compilar o modelo
model_nn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo com as saídas do SVM como rótulos
model_nn.fit(X_train, y_train_cat, epochs=10, validation_data=(X_test, y_test_cat))

# Converter o modelo Keras para TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model_nn)
# Quantização (opcional) para otimização em dispositivos móveis
# converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# Salvar o modelo TFLite
tflite_path = os.path.join(os.path.dirname(SVM_PATH), "model2.tflite")
with open(tflite_path, "wb") as f:
    f.write(tflite_model)

print("Modelo salvo como model.tflite em:", tflite_path)


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score

# Carregar o modelo .tflite salvo na mesma pasta que o SVM_PATH
tflite_path = os.path.join(os.path.dirname(SVM_PATH), "model2.tflite")
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Obter informações sobre os tensores de entrada e saída do modelo TFLite
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Função para fazer previsões com o modelo TFLite
def predict_tflite(input_data):
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

# Previsões com o modelo SVM para comparação
svm_predictions = svm_model.predict(X_test)

# Previsões com o modelo TFLite
tflite_predictions = []
for i in range(len(X_test)):
    input_data = np.array([X_test[i]], dtype=np.float32)
    tflite_pred = predict_tflite(input_data)
    tflite_predictions.append(np.argmax(tflite_pred))  # Obter a classe de maior probabilidade

# Converter `y_test` para índices inteiros usando `CLASS_TO_IDX`
y_test_numeric = np.array([CLASS_TO_IDX[label] for label in y_test])

# Comparar previsões dos dois modelos e calcular precisão
print("Primeiras 10 previsões do SVM:", svm_predictions[:10])
print("Primeiras 10 previsões do modelo TFLite:", tflite_predictions[:10])

# Calcular a precisão do modelo TFLite
tflite_accuracy = accuracy_score(y_test_numeric, tflite_predictions)
print("Precisão do modelo TFLite:", tflite_accuracy)


In [ ]:
import tensorflow as tf
import os

# ... (seu código principal) ...

# Carregue o modelo .tflite usando o caminho definido no seu código
tflite_path = os.path.join(os.path.dirname(SVM_PATH), "model2.tflite")
interpreter = tf.lite.Interpreter(model_path=tflite_path)
interpreter.allocate_tensors()

# Obtenha os detalhes do tensor de entrada
input_details = interpreter.get_input_details()

# Obtenha os detalhes do tensor de saída
output_details = interpreter.get_output_details()

# Imprima as formas dos tensores de entrada e saída
print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])

# ... (restante do seu código principal) ...

![image](https://drive.google.com/uc?export=view&id=1KTqSyyRhemUhoD-1naNKiePZXt8Vfu1j)